In [1]:
!pip install -q transformers bitsandbytes peft trl accelerate xformers wandb datasets einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

# Importing Dependencies

In [2]:
#importing library
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, TextStreamer
from peft import LoraConfig, get_peft_model, PeftModel
import torch, os, wandb
#wandb.login(key = "wandb authorization key")
new_model = "/content/Jyotiyadav/NER-Phi-2"
base_model = "microsoft/phi-1_5"

In [ ]:
#loading the model and tokenizer
bitsandbytes= BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model, device_map={"":0},
    quantization_config= bitsandbytes, trust_remote_code= True
)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#run = wandb.init(project='Fine tuning microsoft phi -1.5', job_type="training", anonymous="allow")

In [ ]:
#Lora Hyperparameter
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["fc1", "fc2","Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 1,426,135,040 || trainable%: 0.5514428703750243


In [ ]:
import pandas as pd
df = pd.read_csv("/content/folder/df_final_llm (1).csv")
df.head()

,email,entities
0,fm door Quinto Di Treviso to HKG / Totali Quan...,"{'pickup_location': ""Quinto Di Treviso Viale d..."
1,", mi quotereste quanto segue per Favore? , Ori...","{'pickup_location': 'Colico LC', 'quantity': '..."
2,"PORT JEBEL ALI Buon pomeriggio, , potreste quo...","{'delivery_port': 'JEBEL ALI Jebel Ali', 'pick..."
3,"d'oglio /cat lai Buon giorno a tutti , Da Quin...","{'pickup_location': ""d'oglio Quinzano d'Oglio""..."
4,CUCINE INOX // LCL ex. Italy to HK .Atelier Gr...,"{'pickup_state': 'Italy Italy Italy', 'deliver..."


In [ ]:
INS = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.
"""

In [ ]:
for i in range(len(df)):
  email = df['email'][i]
  output = df['entities'][i]
  prompt = f"""{INS} ### Input:{email} ### Response:{output}"""
  df['email'].iloc[i] = prompt
df.drop(columns=['entities'], inplace=True)
df.head()
df.rename(columns={'email': 'text'}, inplace=True)

data = Dataset.from_pandas(df)
data

Dataset({
    features: ['text'],
    num_rows: 703
})

In [ ]:
#Tokenzing the dataset
def tok(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True)
    return model_inps

tokenized_training_data = data.map(tok, batched=True,batch_size=32)
tokenized_training_data

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 703
})

In [ ]:
#Training hyperparamters
training_arguments = TrainingArguments(
        output_dir="output",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        #EvaluationStrategy = "steps",
        save_strategy="epoch",
        logging_steps=30,
        max_steps=2000,
        num_train_epochs=2,
        #report_to="wandb"
    )
trainer = Trainer(
    model=model,
    train_dataset=tokenized_training_data["input_ids"],
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: Currently logged in as: jyotiyadav79811 (ml-thesis). Use `wandb login --relogin` to force relogin


Step,Training Loss
30,2.089200
60,1.341600
90,1.238200
120,1.312400
150,1.333700
180,1.105700
210,1.142800
240,1.290500
270,1.073600
300,1.098300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2000, training_loss=0.8674128103256226, metrics={'train_runtime': 378.3381, 'train_samples_per_second': 5.286, 'train_steps_per_second': 5.286, 'total_flos': 2.227883054186496e+16, 'train_loss': 0.8674128103256226, 'epoch': 2.844950213371266})

In [ ]:
#Training

trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2048)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x PhiDecoderLayer(
            (self_attn): PhiSdpaAttention(
              (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
              (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
              (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
              (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
              (rotary_emb): PhiRotaryEmbedding()
            )
            (mlp): PhiMLP(
              (activation_fn): NewGELUActivation()
              (fc1): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=8192, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=

In [ ]:
#Testing the trained model
def phi_stream(prompt):
  runtimeFlag = "cuda:0"
  inputs = tokenizer(f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{prompt}.\n\n### Response:\n ''', return_tensors="pt", return_attention_mask=False).to(runtimeFlag)
  streamer = TextStreamer(tokenizer, skip_prompt= True)
  _ = model.generate(**inputs, streamer=streamer, max_new_tokens=500)
phi_stream("what is large language model")


### Input:data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data/data

In [ ]:
#merging the adpater and pretrained model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= {"": 0},
)
model = PeftModel.from_pretrained(base_model,new_model)
model = model.merge_and_unload()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
model.save_pretrained(new_model)

In [ ]:
tokenizer.save_pretrained(new_model)

('/content/Jyotiyadav/NER-Phi-2/tokenizer_config.json',
 '/content/Jyotiyadav/NER-Phi-2/special_tokens_map.json',
 '/content/Jyotiyadav/NER-Phi-2/vocab.json',
 '/content/Jyotiyadav/NER-Phi-2/merges.txt',
 '/content/Jyotiyadav/NER-Phi-2/added_tokens.json',
 '/content/Jyotiyadav/NER-Phi-2/tokenizer.json')

In [ ]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/Jyotiyadav/NER-Phi-2'. Use `repo_type` argument if needed.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [6]:
model="DataIntelligenceTeam/model-NER-Phi"

In [7]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = model,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [9]:
prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Instruction:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input:
dubai Dear Team, ; Please quote SEA option for below inquiry ; POD :Jebel Ali . ; estimated packing details ;  3300x950x1350 mm ; 3800x1700x1350 mm ; 1900x2000x1200 mm ; 1900x2000x1200 mm ; 2600x1300x1200 mm ; 3900x1100x1350 mm ; 820x780x950 mm ; Total weight kg 5000 approx ; Not stackable ; Marrone Srl ; via Rui, 5 - Loc. Prà dei Risi ; I - 33080 Zoppola (PN) ; Warehouse: 08:00 – 12:00 / 13:00 – 16:30 ; Warehouse Handling + Free days  w.e.f  01-01-2024 ; AED 250 + vat until 10 CBM -  AED 350 + Vat 11 CBM on wards ; Free time applicable - 7 days only for General cargo from DO readiness & No free time for HAZ CARGO, post which standard storage tariff will apply. ; Important Notice ;  : Considering the ongoing circumstances, please be informed after booking confirmation the rates and schedules may undergo changes without advance notice. ; Please also expect vessel delays and changes in transit time. ; CSS will not be liable for any claims that arise due to this. ; Deepak Unnikrishnan ; Sr Sales Coordinator ; Consolidated Shipping Service L.L.C. ; Office: +971 4 883 1303 | Ext: 1163| Toll Free: 800277 | Mobile: +971501513697| ; Dubai | United Arab Emirates ; nvo-import5@cssdubai.com | www.cssgroupsite.com ; Our Offices: Dubai | Abu Dhabi | Sharjah | Ras-Al-Khaimah | Bahrain | Oman | Qatar | Saudi Arabia | Kuwait | Iraq | Africa | Turkiye | India | Sri Lanka | ; Disclaimer: This email and any attachments are confidential and may also be privileged. If you are not the intended recipient, please delete all copies and notify the sender immediately. ; Please read our ; disclaimer.

### Response:
'''

result = pipe(
    f"{prompt}",
    do_sample=True,
    max_new_tokens=1000,
    temperature=0.1,
    top_k=0,
    top_p=0,
    num_return_sequences=1,
)
print(result[0]['generated_text'])


Your task is to extract the information corresponding to the provided labels from the below given email.

### Instruction:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volum